# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "C:/Users/empha/Desktop/Data_Analytics/Semana_7/python-api-challenge/APIChallengePy/WeatherPy/cities.csv"
cities_df = pd.read_csv(csv_path)
cities_df_cleaned = cities_df.loc[cities_df["Date"]!=0,:]
cities_df_cleaned.head()


,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,1,US,1619619301,62,34.1478,-119.1951,59.00,4.61
1,1,100,ID,1619619301,96,-2.6350,115.2274,74.25,1.52
3,3,49,RU,1619619302,97,69.3000,139.9000,3.31,6.11
4,4,75,US,1619619303,94,22.0752,-159.3190,66.99,8.05
5,5,29,VU,1619619303,63,-19.5500,169.2667,73.29,12.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Test con solo 10 records
prueba_df = cities_df_cleaned.head(10)
lat = prueba_df["Lat"].tolist()
lng = prueba_df["Lng"].tolist()
weight = prueba_df["Humidity"].astype(float).to_list()
coordinates = [(lat[x],lng[x]) for x in range(len(lat))]
figure_layout = {
    "width":"600px",
    "height":"500px",
    "border":"1px solid black",
    "padding":"1px",
    "margin":"0 auto 0 auto"
}
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(coordinates,weights=weight))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
lat = cities_df_cleaned["Lat"].tolist()
lng = cities_df_cleaned["Lng"].tolist()
weight = cities_df_cleaned["Humidity"].astype(float).to_list()
coordinates = [(lat[x],lng[x]) for x in range(len(lat))]
figure_layout = {
    "width":"600px",
    "height":"500px",
    "border":"1px solid black",
    "padding":"1px",
    "margin":"0 auto 0 auto"
}
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(coordinates,weights=weight))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = cities_df_cleaned.loc[(cities_df_cleaned["Max Temp"]>=70) & (cities_df_cleaned["Max Temp"]<80),:]
hotel_df = hotel_df.loc[hotel_df["Wind Speed"]<10,:]
hotel_df = hotel_df.loc[hotel_df["Cloudiness"]==0,:]
hotel_df

,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
29,29,0,BR,1619619315,60,-20.4725,-45.1256,77.67,2.86
156,156,0,CN,1619619382,13,38.4167,77.2406,72.01,0.96
210,210,0,CL,1619619410,61,-22.0920,-70.1979,71.15,7.74
220,220,0,DE,1619619416,32,48.7267,9.2514,70.00,9.22
241,241,0,IL,1619619427,49,31.6100,34.7642,75.99,3.00
285,285,0,GE,1619619450,34,41.9563,42.9566,78.80,8.05
342,342,0,BR,1619619481,43,-20.4167,-54.0667,73.40,6.91
430,430,0,NC,1619619530,82,-22.2763,166.4572,75.00,9.69
555,555,0,CN,1619619599,13,41.1231,80.2644,75.31,3.15
585,585,0,CN,1619619616,12,39.4547,75.9797,75.20,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ''
hotel_df["Hotel Lat"] = ''
hotel_df["Hotel Lng"] = ''
hotel_df["City"] = ''
lat_list = hotel_df["Lat"].tolist()
lng_list = hotel_df["Lng"].tolist()
string_cord = [str(lat_list[x])+", "+str(lng_list[x]) for x in range(len(lat_list))]
radius = 5000
hotel_df


,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng,City
29,29,0,BR,1619619315,60,-20.4725,-45.1256,77.67,2.86,,,,
156,156,0,CN,1619619382,13,38.4167,77.2406,72.01,0.96,,,,
210,210,0,CL,1619619410,61,-22.0920,-70.1979,71.15,7.74,,,,
220,220,0,DE,1619619416,32,48.7267,9.2514,70.00,9.22,,,,
241,241,0,IL,1619619427,49,31.6100,34.7642,75.99,3.00,,,,
285,285,0,GE,1619619450,34,41.9563,42.9566,78.80,8.05,,,,
342,342,0,BR,1619619481,43,-20.4167,-54.0667,73.40,6.91,,,,
430,430,0,NC,1619619530,82,-22.2763,166.4572,75.00,9.69,,,,
555,555,0,CN,1619619599,13,41.1231,80.2644,75.31,3.15,,,,
585,585,0,CN,1619619616,12,39.4547,75.9797,75.20,2.24,,,,


In [7]:
hotel_names = []
hotel_lat = []
hotel_lng = []
hotel_city = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for coord in string_cord:
    params = {
    "location":coord,
    "radius":radius,
    "keyword":"hotel",
    "key":g_key
    }
    data_hotels = requests.get(base_url,params=params).json()
    try:
        print("Hotel Name: "+str(data_hotels["results"][0]["name"]))
        print("Vicinity: "+data_hotels["results"][0]["vicinity"])
        print("Lat: "+str(data_hotels["results"][0]["geometry"]["location"]["lat"]))
        print("Lng: "+str(data_hotels["results"][0]["geometry"]["location"]["lng"]))
        hotel_names.append(data_hotels["results"][0]["name"])
        hotel_lat.append(data_hotels["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(data_hotels["results"][0]["geometry"]["location"]["lng"])
        hotel_city.append(data_hotels["results"][0]["vicinity"])
    except IndexError:
        print("No hotel available")
        hotel_names.append("NaN")
        hotel_lat.append("NaN")
        hotel_lng.append("NaN")
        hotel_city.append("NaN")

Hotel Name: Hotel Fazenda Palestina
Vicinity: Fazenda Palestina Zona Rural, Itapecerica
Lat: -20.4625473
Lng: -45.102169
Hotel Name: Xinsheng Hotel
Vicinity: 4 Xincheng Rd, Yarkant County, Kashgar Prefecture
Lat: 38.414222
Lng: 77.247519
Hotel Name: Hostal Paraiso
Vicinity: Calama 2943, Tocopilla
Lat: -22.0800553
Lng: -70.1932312
Hotel Name: Hotel Ochsen
Vicinity: Ulmer Str. 323, Stuttgart
Lat: 48.773642
Lng: 9.241589
Hotel Name: Desert Gat
Vicinity: Sderot HaAtsma'ut 134, Kiryat Gat
Lat: 31.6027431
Lng: 34.7643394
No hotel available
No hotel available
Hotel Name: Hilton Noumea La Promenade Residences
Vicinity: 109 Promenade Roger Laroque, Nouméa
Lat: -22.3018092
Lng: 166.446066
Hotel Name: Jianshe Hotel
Vicinity: Aksu City, Tazhong Rd, 塔中路
Lat: 41.161712
Lng: 80.28168939999999
Hotel Name: Radisson Blu Hotel Kashgar
Vicinity: No. 2, Duolaitebage Road, Kashgar Prefecture
Lat: 39.44799
Lng: 76.00312


In [8]:
hotel_df["Hotel Name"] = hotel_names
hotel_df["Hotel Lat"] = hotel_lat
hotel_df["Hotel Lng"] = hotel_lng
hotel_df["City"] = hotel_city



In [9]:
hotel_df = hotel_df.loc[hotel_df["Hotel Lat"]!="NaN",:]
hotel_df

,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng,City
29,29,0,BR,1619619315,60,-20.4725,-45.1256,77.67,2.86,Hotel Fazenda Palestina,-20.4625,-45.1022,"Fazenda Palestina Zona Rural, Itapecerica"
156,156,0,CN,1619619382,13,38.4167,77.2406,72.01,0.96,Xinsheng Hotel,38.4142,77.2475,"4 Xincheng Rd, Yarkant County, Kashgar Prefecture"
210,210,0,CL,1619619410,61,-22.0920,-70.1979,71.15,7.74,Hostal Paraiso,-22.0801,-70.1932,"Calama 2943, Tocopilla"
220,220,0,DE,1619619416,32,48.7267,9.2514,70.00,9.22,Hotel Ochsen,48.7736,9.24159,"Ulmer Str. 323, Stuttgart"
241,241,0,IL,1619619427,49,31.6100,34.7642,75.99,3.00,Desert Gat,31.6027,34.7643,"Sderot HaAtsma'ut 134, Kiryat Gat"
430,430,0,NC,1619619530,82,-22.2763,166.4572,75.00,9.69,Hilton Noumea La Promenade Residences,-22.3018,166.446,"109 Promenade Roger Laroque, Nouméa"
555,555,0,CN,1619619599,13,41.1231,80.2644,75.31,3.15,Jianshe Hotel,41.1617,80.2817,"Aksu City, Tazhong Rd, 塔中路"
585,585,0,CN,1619619616,12,39.4547,75.9797,75.20,2.24,Radisson Blu Hotel Kashgar,39.448,76.0031,"No. 2, Duolaitebage Road, Kashgar Prefecture"


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Hotel Fazenda Palestina</dd>\n<dt>City</dt><dd>Fazenda Palestina Zona Rural, Itapecerica</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Xinsheng Hotel</dd>\n<dt>City</dt><dd>4 Xincheng Rd, Yarkant County, Kashgar Prefecture</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hostal Paraiso</dd>\n<dt>City</dt><dd>Calama 2943, Tocopilla</dd>\n<dt>Country</dt><dd>CL</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Ochsen</dd>\n<dt>City</dt><dd>Ulmer Str. 323, Stuttgart</dd>\n<dt>Country</dt><dd>DE</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Desert Gat</dd>\n<dt>City</dt><dd>Sderot HaAtsma'ut 134, Kiryat Gat</dd>\n<dt>Country</dt><dd>IL</dd>\n</dl>\n",
 '\n<dl>\n<dt>Name</dt><dd>Hilton Noumea La Promenade Residences</dd>\n<dt>City</dt><dd>109 Promenade Roger Laroque, Nouméa</dd>\n<dt>Country</dt><dd>NC</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Jianshe Hotel</dd>\n<dt>City</dt><dd>Aksu City, Tazhong Rd, 塔中路</dd>\n<dt>Co

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…